## **Trabajo Práctico Individual n° 2: Aprendizaje Supervisado**
____
__Universidad Tecnológica Nacional, Buenos Aires__\
__Ingeniería Industrial__\
__Ciencia de Datos - Curso I5521__
____

- Nombre: Guido
- Apellido: Cambieri
- Legajo: 141509

En este trabajo práctico aplicaremos algunas técnicas aprendidas para el aprendizaje automático supervisado.

Utilizaremos un dataset de airbnb donde buscaremos predecir el precio de las noches de cada hogar en funcion de algunas variables.

#### Variables:
- Unnamed: 0: Variable de contador de registros (no será utilizada en nuetro análisis)
- neighbourhood: Barrio donde se encuentra ubicado la casa.
- room_type: Tipo de hogar en alquiler.
- price: Precio por noche.
- minimum_nights: Cantidad minimas de noches que pueden alquilar el hogar.
- number_of_reviews: Cantidad de reviews
- reviews_per_month: Tasa de reviews que tienen por mes
- calculated_host_listings_count: Cantidad de hogares ofrecidos por el host.
- availability_365: Cantidad de dias que se encuentra en alquiler el hogar

## **Objetivos del trabajo practico**
- resolver los requisitos que se detallan en las proximas celdas
- dejar comentarios # en cada linea de codigo que se agregue
- fecha limite de entrega: 13 de Octubre 2022 a las 23h59

In [1]:
## Importo librerías de maniupulación de datos
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
## Importo librerías de Aprendizaje automático
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV

##### **Objetivo 1**
Lo primero que deberan hacer será importar
- el dataset bajo en nombre "london"
- corroborar que lo hayamos cargado correctamente.

In [3]:
london = pd.read_csv(r'C:\Users\Pillud\Documents\UTN\Ciencia de Datos\TP2\TP2_airbnb_london.csv')
print(london.head()) # Imprimo las 5 primeras filas del dataframe(df) para ver que se cargo correctamente

   Unnamed: 0  neighbourhood        room_type  price  minimum_nights  \
0           0       Haringey     Private room     35               1   
1           1         Ealing     Private room     70               2   
2           2      Islington     Private room     45               1   
3           5  Tower Hamlets  Entire home/apt    102               5   
4           6         Barnet     Private room     29               5   

   number_of_reviews  reviews_per_month  calculated_host_listings_count  \
0                131               1.71                               4   
1                  2               0.07                               6   
2                 14               0.14                               2   
3                  7               0.07                              56   
4                114               1.13                               3   

   availability_365  
0               262  
1                62  
2               364  
3               295  
4     

In [4]:
#veo cantida de filas y columnas
london.shape

(51827, 9)

In [5]:
#descripcion general de la data
london.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51827 entries, 0 to 51826
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Unnamed: 0                      51827 non-null  int64  
 1   neighbourhood                   51827 non-null  object 
 2   room_type                       51827 non-null  object 
 3   price                           51827 non-null  int64  
 4   minimum_nights                  51827 non-null  int64  
 5   number_of_reviews               51827 non-null  int64  
 6   reviews_per_month               51827 non-null  float64
 7   calculated_host_listings_count  51827 non-null  int64  
 8   availability_365                51827 non-null  int64  
dtypes: float64(1), int64(6), object(2)
memory usage: 3.6+ MB


##### **Objetivo 2**
Ahora imprimir también las dimensiones del dataset y eliminar la variable "Unnamed:0" del dataset.
Volver a guardar el dataset sin esta variable bajo el nombre "london"

In [6]:
print("Dimensiones:", london.shape)
london = london.drop(columns=['Unnamed: 0'])

Dimensiones: (51827, 9)


In [7]:
#verifico que se haya borrado la columna
london.shape

(51827, 8)

In [8]:
#verifico que se haya borrado la columna
london.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51827 entries, 0 to 51826
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   neighbourhood                   51827 non-null  object 
 1   room_type                       51827 non-null  object 
 2   price                           51827 non-null  int64  
 3   minimum_nights                  51827 non-null  int64  
 4   number_of_reviews               51827 non-null  int64  
 5   reviews_per_month               51827 non-null  float64
 6   calculated_host_listings_count  51827 non-null  int64  
 7   availability_365                51827 non-null  int64  
dtypes: float64(1), int64(5), object(2)
memory usage: 3.2+ MB


##### **Objetivo 3**
Corroborar que el dataset no cuenta con valores nulos, si los hubiera llenarlos todos con 0.

In [9]:
print("Valores nulos en el dataset:\n", london.isnull().sum())

Valores nulos en el dataset:
 neighbourhood                     0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64


In [10]:
#london.fillna(0, inplace=True) #no hace falta porque no hay nulos

##### **Objetivo 4**
Mostrar el tipo de dato para las variables "neighbourhood" y "room_type".

In [11]:
print("neighbourhood", london['neighbourhood'].dtype)
print("room_type", london['room_type'].dtype)

neighbourhood object
room_type object


##### **Objetivo 5**
Generar las variables dummies para estas dos variables categóricas y unirlas al dataset london.


In [12]:
## Genero las dummies para la variable neighbourhood, dentro de la variable neigh_dummies.
neigh_dummies = pd.get_dummies(london['neighbourhood'])

Imprimir las dimensiones y primeras lineas de estas dummies

In [13]:
print(neigh_dummies.head())

   Barking and Dagenham  Barnet  Bexley  Brent  Bromley  Camden  \
0                 False   False   False  False    False   False   
1                 False   False   False  False    False   False   
2                 False   False   False  False    False   False   
3                 False   False   False  False    False   False   
4                 False    True   False  False    False   False   

   City of London  Croydon  Ealing  Enfield  ...  Merton  Newham  Redbridge  \
0           False    False   False    False  ...   False   False      False   
1           False    False    True    False  ...   False   False      False   
2           False    False   False    False  ...   False   False      False   
3           False    False   False    False  ...   False   False      False   
4           False    False   False    False  ...   False   False      False   

   Richmond upon Thames  Southwark  Sutton  Tower Hamlets  Waltham Forest  \
0                 False      False   False   

In [14]:
## Genero las dummies para la variable room_type, dentro de la variable room_dummies.
room_dummies = pd.get_dummies(london['room_type'])

Imprimir las dimensiones y primeras lineas de estas dummies

In [15]:
print(room_dummies.head())

   Entire home/apt  Private room  Shared room
0            False          True        False
1            False          True        False
2            False          True        False
3             True         False        False
4            False          True        False


Joinear las variables dummies al dataset



In [16]:
#london = london.join(room_dummies,neigh_dummies) #me tira error asi
london = pd.concat([london, neigh_dummies, room_dummies], axis=1) #axis=1: Une los DataFrames horizontalmente (agregando nuevas columnas)



##### Dividiremos nuestro dataset en la variable dependiente "y" (label o etiqueta) y las variables independientes "x"

In [17]:

y = np.array(london[["price"]])
x = london.drop(['price', 'neighbourhood','room_type'], axis=1)

##### **Objetivo 7**

Dividir el dataset entre Train y Test, tomar un test_size del 30%.

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30)

##### **Objetivo 8**
Con el StandardScaler generado en la linea siguiente, realizar el transform para el xtrain y xtest guardandolos en nuevas variables llamadas "xtrain_scal" y "xtest_scal".


In [19]:
scaler = preprocessing.StandardScaler().fit(x_train)

In [20]:
xtrain_scal = scaler.transform(x_train)

In [21]:
xtest_scal = scaler.transform(x_test)

##### **Objetivo 9**
Definir un modelo de regresion lineal standard y entrenarlo para nuestro dataset (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

Obtener las predicciones y el "mean_squared_error" para el test set.

In [22]:
#defino el modelo
modelo_LR = LinearRegression()
modelo_LR.fit(xtrain_scal, y_train)

LinearRegression()

In [23]:
#ahora obtengo las predicciones sobre el conjunto de prueba
y_pred = modelo_LR.predict(xtest_scal)

In [24]:
#calculo el MSE y lo muestro
MSE = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (MSE):", MSE)

Mean Squared Error (MSE): 1544.2074822972356


In [25]:
# para comaprar veo la media del precio del dataset
price_mean = london["price"].mean()
print("Media del campo 'price':", price_mean)

Media del campo 'price': 87.03878287379165


In [26]:
# evidentemente el modelo no predice bien, hay mucha variabildiad

##### **Objetivo 10**
Definir un modelo de regresion SVM Regressor (https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html) e implementar GridSearch para realizar la búsqueda de hiperparámetros.

Asignar dos posibles valores para el parámetro "kernel", y tres posibles valores para "C" y "gamma".

Setear al parámetro "refit" en True y hacer 5 CrossValidation.

In [27]:
svr_model = SVR()

In [28]:
parametros_grid = {
    'kernel': ['linear'],
    'C': [0.1, 1, 10],
    'gamma': [0.01, 0.1, 1]
}

In [29]:
grid_search = GridSearchCV(
    estimator=svr_model,
    param_grid=parametros_grid,
    scoring='neg_mean_squared_error',
    cv=5,
    refit=True
)

In [30]:
grid_search.fit(xtrain_scal, y_train.ravel())

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [0.1, 1, 10], 'gamma': [0.01, 0.1, 1],
                         'kernel': ['linear']},
             scoring='neg_mean_squared_error')

In [32]:
mejores_parametros = grid_search.best_params_
print(mejores_parametros)

{'C': 10, 'gamma': 0.01, 'kernel': 'linear'}


##### A

##### **Objetivo 11**

Mostrar cuales fueron los mejores hiperparámetros y para estos obtener el mean_squared_error en el test set del punto anterior

In [34]:
mejores_parametros = grid_search.best_params_
print(mejores_parametros)

{'C': 10, 'gamma': 0.01, 'kernel': 'linear'}


In [35]:
#Hago una prediccion el conjunto de prueba usando los mejores hiperparam
mejor_modelo = grid_search.best_estimator_  
y_pred = mejor_modelo.predict(xtest_scal)

In [36]:
# MSE del conjunto de prueba
MSE_test = mean_squared_error(y_test, y_pred)
print("MSE del conjunto de prueba:", MSE_test)

MSE del conjunto de prueba: 1638.9819713835577


##### **Objetivo 12**
Repetir el objetivo 10 pero ahora seteando nuevos valores para los hiperparámetros mencionados con el objetivo de mejorar las predicciones.

Cuales son los mejores hiperparámetros ahora?
Obtenemos una mejora en las predicciones?
Mostrar los resultados.

In [41]:
svr_mejor_modelo = SVR()

In [42]:
parametros_grid = {
    'kernel': ['rbf'],
    'C': [0.1, 1, 10],
    'gamma': [0.01, 0.1, 1]
}

In [43]:
grid_search = GridSearchCV(
    estimator=svr_mejor_modelo,
    param_grid=parametros_grid,
    scoring='neg_mean_squared_error',
    cv=5,
    refit=True
)

In [44]:
grid_search.fit(xtrain_scal, y_train.ravel())

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [0.1, 1, 10], 'gamma': [0.01, 0.1, 1],
                         'kernel': ['rbf']},
             scoring='neg_mean_squared_error')

In [45]:
mejores_parametros = grid_search.best_params_
print(mejores_parametros)

{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}


In [46]:
#Hago una prediccion el conjunto de prueba usando los mejores hiperparam
mejor_modelo_2 = grid_search.best_estimator_  
y_pred = mejor_modelo_2.predict(xtest_scal)

In [47]:
# MSE del conjunto de prueba
MSE_test = mean_squared_error(y_test, y_pred)
print("MSE del conjunto de prueba:", MSE_test)

MSE del conjunto de prueba: 1576.0044313980382


In [ ]:
# los mejores parametros seria {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'} ya que presentan un menor MSE contra {'C': 10, 'gamma': 0.01, 'kernel': 'linear'}
# anque el mejor de los 3, es la Regresion Lineal simple